In [2]:
import pandas as pd
import time
from datetime import date
from datetime import datetime

Задание 1

In [3]:
data = pd.read_csv('ml-latest-small/ratings.csv')

In [4]:
# def groupby_function(data):
#     return data.timestamp.max() - data.timestamp.min()

# data.groupby('userId').apply(groupby_function)


lifetime = data.groupby(data.userId).max()[['timestamp']] - data.groupby(data.userId).min()[['timestamp']]

data_100 = data.groupby(data.userId).count()[['rating']].query('rating > {}'.format(100))

datetime.fromtimestamp(data_100.join(lifetime).mean()[['timestamp']]) - datetime(1970,1,1,0,0,0)

datetime.timedelta(days=464, seconds=1707)

Задание 2

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [6]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [7]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [8]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [9]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
all_tab = rzd.merge(auto, how='outer')
all_tab = all_tab.merge(air,  how='outer')
all_tab

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [15]:
client_base.merge(all_tab, how='outer')

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Данные добавлял бы в один столбце, чтобы при обращении сразу все координаты были. Важно отметить, что координата должна привязываться именно к визиту/покупке, а не к пользователю. 
Пользователь может производить покупки в разных  точках. К этому хорошо бы добавить, время и сумму покупки. 

Как вариант это позволит нам понимать, как меняются координаты в зависимости от времени суток. Один из вариантов, как меняются заказы из офиса и дома.

2. К визитам привязывать координаты, а каждому пользователю привязаны визиты. 
Дополнительно фиксировать покупку и добавление в корзнику без последующей покупки. 

3. Не сильно должно чего-то поменяться. 

4. Координаты без дополнительной информации не достаточно информативны, но при наличии достаточного объема данных можем: 

4.1. Узнать, на работе сделан заказ или дома.

4.2. Регулярность заказов из определнного места положения. Или только просматривают без заказов.

4.3. Зная средний чек, можно менять рекомендации. В центре Москвы ср. чек может быть выше. 

Первая гипотеза, что пришло в голову доставка еды, в центре Москвы средний чек вечером выше, чем днем. Днем ланчи в офис заказывают, вечером ужин домой.

